In [1]:
!pip install pulp
!pip install ortools

    100% |████████████████████████████████| 13.6MB 2.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/3b/f1/f5/f6d0aac56dd154c2593d3fa5e901aa9bc32a51bd8594b4bf8e
Successfully built pulp
    100% |████████████████████████████████| 25.6MB 762kB/s 


In [0]:
import math
import matplotlib.pyplot as plt
import numpy as np


# Linear programming problem

-problem

- target function to optimize
100x + 100 y

- constraints
 - x + 2y <=16
 - 3x + y <= 18
 - x >0
 - y>0

PuLP

In [0]:
import pulp

In [6]:
# Problem 1

# Numerical model
m = pulp.LpProblem(sense=pulp.LpMaximize) 

# valiables to optimize
x = pulp.LpVariable('x', lowBound=0) 
y = pulp.LpVariable('y', lowBound=0) 

# Objective function to optimize ( maximize)
m += 100 * x + 100 * y

#  Constraints
m += x + 2 * y <= 16 
m += 3 * x + y <= 18 

# Solve
m.solve()
print(pulp.value(x), pulp.value(y)) # 4, 6



4.0 6.0


In [9]:
## problem2 supply optimization 

from itertools import product


# define data
np.random.seed(1)
nw, nf = 3, 4

pr = list(product(range(nw),range(nf)))
supply = np.random.randint(30, 50, nw)
demand = np.random.randint(20, 40, nf)
cost = np.random.randint(10, 20, (nw,nf))

print('cost')
print(cost)

print('supply lower limit')
print(supply)

print('demand lower limit')
print(demand)

# Modeling
m1 = pulp.LpProblem()

# variables ()
v1 = {(i,j):pulp.LpVariable('v%d_%d'%(i,j), lowBound=0) for i,j in pr}

# Object function
m1 += pulp.lpSum(cost[i][j] * v1[i,j] for i,j in pr)

# Constraints
for i in range(nw):
    m1 += pulp.lpSum(v1[i,j] for j in range(nf)) <= supply[i]
for j in range(nf):
    m1 += pulp.lpSum(v1[i,j] for i in range(nw)) >= demand[j]

# solve    
m1.solve()
{k:pulp.value(x) for k,x in v1.items() if pulp.value(x) > 0}


cost
[[10 10 11 17]
 [16 19 12 14]
 [15 12 14 12]]
supply lower limit
[35 41 42]
demand lower limit
[28 29 31 25]


{(0, 0): 28.0,
 (0, 1): 7.0,
 (1, 2): 31.0,
 (1, 3): 5.0,
 (2, 1): 22.0,
 (2, 3): 20.0}

Scipy

In [19]:
from scipy.optimize import fmin,minimize

# Problem1


# Objective
def func(X):
    return -(100*X[0]+100*X[1])

# constraint function
def cons1(X):
    return 16-(X[0] + 2*X[1])
def cons2(X):
    return 18-(3*X[0] + X[1])
def cons3(X):
    return np.abs(X)

# set constraits (negative constraint)
cons = (
    {'type': 'ineq', 'fun': cons1},
    {'type': 'ineq', 'fun': cons2},
    {'type': 'ineq', 'fun': cons3},
)
X = [1.0,1.0] #init

#result = minimize(func, x0=X, constraints=cons, method="SLSQP")
result = minimize(func, x0=X, constraints=cons, method="COBYLA")

print(result)

     fun: -1000.0
   maxcv: 3.552713678800501e-15
 message: 'Optimization terminated successfully.'
    nfev: 18
  status: 1
 success: True
       x: array([4., 6.])


ortools

In [20]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('SolveIntegerProblem',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


# Problem1

# set variables
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, solver.infinity(), 'x')
y = solver.IntVar(0.0, solver.infinity(), 'y')


# constraints
constraint1 = solver.Constraint(-solver.infinity(), 16)
constraint1.SetCoefficient(x, 1)
constraint1.SetCoefficient(y, 2)

constraint2 = solver.Constraint(-solver.infinity(), 18)
constraint2.SetCoefficient(x, 3)
constraint2.SetCoefficient(y, 1)

# Objective  .
objective = solver.Objective()
objective.SetCoefficient(x, 100)
objective.SetCoefficient(y, 100)
objective.SetMaximization()

"""Solve the problem and print the solution."""
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

# The solution looks legit (when using solvers other than
# GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
assert solver.VerifySolution(1e-7, True)

print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

# The objective value of the solution.
print('Optimal objective value = %d' % solver.Objective().Value())
print()
# The value of each variable in the solution.
variable_list = [x, y]

for variable in variable_list:
  print('%s = %d' % (variable.name(), variable.solution_value()))



Number of variables = 2
Number of constraints = 2
Optimal objective value = 1000

x = 4
y = 6


# Non linear optimization

In [0]:

# Non linear

# Objective
def func(x):
    return x ** 2

# constraint function
def cons(x):
    return -(x + 1)

# set constraits (negative constraint)
cons = (
    {'type': 'ineq', 'fun': cons}
)
x = -10 #init

result = minimize(func, x0=x, constraints=cons, method="SLSQP")
print(result)

     fun: 0.9999999999996732
     jac: array([-1.99999999])
 message: 'Optimization terminated successfully.'
    nfev: 6
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([-1.])


In [0]:
list(zip(range(nw),range(nf)))


[(0, 0), (1, 1), (2, 2)]